In [2]:
import pandas as pd
from dateutil import relativedelta, parser
from pandas.tseries.offsets import BDay 
from datetime import datetime, date, time
import numpy as np
import pickle
import time
import re
import matplotlib.pyplot as plt
import pandas_datareader.data as reader
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/with_all_ft_clean.pkl")
df.head()

,texts,pct_change1,pct_change2,beta1,beta2,mkt_excess1,mkt_excess2,item 1.01,item 1.02,item 1.03,item 1.04,item 2.01,item 2.02,item 2.03,item 2.04,item 2.05,item 2.06,item 2.10,item 3.01,item 3.02,item 3.03,item 4.01,item 4.02,item 4.04,item 5.01,item 5.02,item 5.03,item 5.04,item 5.05,item 5.07,item 5.08,item 7.01,item 7.02,item 8.01,item 9.01,item 9.02,VIX,label1,label2,SHRCD_12.0,SHRCD_14.0,SHRCD_18.0,SHRCD_21.0,SHRCD_31.0,SHRCD_41.0,SHRCD_44.0,SHRCD_48.0,SHRCD_71.0,SHRCD_72.0,SHRCD_73.0,SHRCD_74.0,ind_21,ind_22,ind_23,ind_31,ind_32,ind_33,ind_42,ind_44,ind_45,ind_48,ind_49,ind_51,ind_52,ind_53,ind_54,ind_55,ind_56,ind_61,ind_62,ind_71,ind_72,ind_81,ind_na,EXCHCD_1.0,EXCHCD_2.0,EXCHCD_3.0,EXCHCD_4.0,Mkt_Cap
0,Item 2.02\nResults of Operations and Financial...,0.039772,0.018824,0.1235,0.12100,0.039502,0.017531,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,86355.36
1,Item 2.02 Results of Operations and Financial ...,0.005894,0.019647,0.9098,0.91085,-0.003435,0.005013,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,20.47,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,125102.08
2,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,-0.019268,-0.019268,0.8742,0.87420,-0.013888,-0.013888,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,19.07,neg,neg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,126812.46
3,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.077505,0.035918,0.8901,0.88485,0.076161,0.027620,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,139273.80
4,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.076013,0.076013,0.9225,0.92250,0.070040,0.070040,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,13.51,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,144649.28


In [6]:
def label1(row):
    if row["pct_change1"] > row["beta1"]*row["mkt_excess1"]:
        return "pos"
    else:
        return "neg"

def label2(row):
    if row["pct_change2"] > row["beta2"]*row["mkt_excess2"]:
        return "pos"
    else:
        return "neg"

In [7]:
df["label1"] = df.apply(label1, axis = 1)
df["label2"] = df.apply(label2, axis = 1)

In [9]:
# Split into train and test data
X_train, X_test, y_train, y_test = train_test_split(df.index, df["label2"],
                                                    stratify=df["label2"], 
                                                    test_size=0.2,
                                                    random_state = 20)

In [11]:
# get column names of categorical features
item_list = [i for i in df.columns if not i.find("item")]
ind_list = [i for i in df.columns if not i.find("ind")]
shrcd_list = [i for i in df.columns if not i.find("SHRCD")]
exc_list = [i for i in df.columns if not i.find("EXCHCD")]

In [13]:
# combine all feature column names
cols = ['VIX', "Mkt_Cap"]
cols.extend(item_list)
cols.extend(ind_list)
cols.extend(shrcd_list)
cols.extend(exc_list)

In [14]:
# get numerical features
num_train = df.loc[X_train, cols]
num_test = df.loc[X_test, cols]

#get texts
text_train = df.loc[X_train, 'texts']
text_test = df.loc[X_test, 'texts']


In [40]:
text_train.values[:5]

array(["Item 5.02\nDeparture of Directors or Certain Officers; Election of Directors; Appointment of Certain Officers; Compensatory Arrangements of Certain Officers.\n  Adoption of 2019 Equity Compensation Plan\n  On June 13, 2019, Pluristem Therapeutics Inc. (the “Company”) held its 2019 Annual Meeting of Stockholders (the “2019 Annual Meeting”). At the 2019 Annual Meeting, the Company’s stockholders approved the Company's 2019 Equity Compensation Plan (the “2019 Equity Compensation Plan”). The 2019 Equity Compensation Plan previously had been approved, subject to stockholder approval, by the Company’s Board of Directors and its Compensation Committee on March 28, 2019.\n  The Company’s officers and directors are among the persons eligible to receive awards under the 2019 Equity Compensation Plan in accordance with the terms and conditions thereunder. A detailed summary of the  2019 Equity Compensation Plan is set forth in the Company’s Definitive Proxy Statement on Schedule 14A for t

In [16]:
num_train.head()

,VIX,Mkt_Cap,item 1.01,item 1.02,item 1.03,item 1.04,item 2.01,item 2.02,item 2.03,item 2.04,item 2.05,item 2.06,item 2.10,item 3.01,item 3.02,item 3.03,item 4.01,item 4.02,item 4.04,item 5.01,item 5.02,item 5.03,item 5.04,item 5.05,item 5.07,item 5.08,item 7.01,item 7.02,item 8.01,item 9.01,item 9.02,ind_21,ind_22,ind_23,ind_31,ind_32,ind_33,ind_42,ind_44,ind_45,ind_48,ind_49,ind_51,ind_52,ind_53,ind_54,ind_55,ind_56,ind_61,ind_62,ind_71,ind_72,ind_81,ind_na,SHRCD_12.0,SHRCD_14.0,SHRCD_18.0,SHRCD_21.0,SHRCD_31.0,SHRCD_41.0,SHRCD_44.0,SHRCD_48.0,SHRCD_71.0,SHRCD_72.0,SHRCD_73.0,SHRCD_74.0,EXCHCD_1.0,EXCHCD_2.0,EXCHCD_3.0,EXCHCD_4.0
11987,15.28,83881.28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9867,15.17,4563169.92,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
16039,14.95,1675050.44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10655,15.82,33693967.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5677,13.50,591050.35,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [17]:
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler()

# standardize volatility index and market cap by mean and standard deviation (standardize train and test sets separately)
num_train["VIX"] = x_scaler.fit_transform(np.array(num_train["VIX"]).reshape(-1,1))
num_test["VIX"] = x_scaler.transform(np.array(num_test["VIX"]).reshape(-1,1))
num_train["Mkt_Cap"] = x_scaler.fit_transform(np.array(num_train["Mkt_Cap"]).reshape(-1,1))
num_test["Mkt_Cap"] = x_scaler.transform(np.array(num_test["Mkt_Cap"]).reshape(-1,1))

In [38]:
num_train.head()

,VIX,Mkt_Cap,item 1.01,item 1.02,item 1.03,item 1.04,item 2.01,item 2.02,item 2.03,item 2.04,item 2.05,item 2.06,item 2.10,item 3.01,item 3.02,item 3.03,item 4.01,item 4.02,item 4.04,item 5.01,item 5.02,item 5.03,item 5.04,item 5.05,item 5.07,item 5.08,item 7.01,item 7.02,item 8.01,item 9.01,item 9.02,ind_21,ind_22,ind_23,ind_31,ind_32,ind_33,ind_42,ind_44,ind_45,ind_48,ind_49,ind_51,ind_52,ind_53,ind_54,ind_55,ind_56,ind_61,ind_62,ind_71,ind_72,ind_81,ind_na,SHRCD_12.0,SHRCD_14.0,SHRCD_18.0,SHRCD_21.0,SHRCD_31.0,SHRCD_41.0,SHRCD_44.0,SHRCD_48.0,SHRCD_71.0,SHRCD_72.0,SHRCD_73.0,SHRCD_74.0,EXCHCD_1.0,EXCHCD_2.0,EXCHCD_3.0,EXCHCD_4.0
11987,-0.226147,-0.248422,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
9867,-0.273114,-0.122885,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
16039,-0.367047,-0.203828,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10655,0.004418,0.693537,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5677,-0.986155,-0.234208,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [22]:
# get a train set data frame of texts and labels only
tt1 = text_train.to_frame().rename({'texts': 'text'}, axis=1)
tt1.loc[:, 'label'] = y_train

In [25]:
tt1.head()

,text,label
11987,Item 5.02\nDeparture of Directors or Certain O...,pos
9867,Item 2.02 Results of Operations and Financial ...,neg
16039,Item 7.01. Regulation FD Disclosure.\nOn May 2...,pos
10655,Item 5.02\nDeparture of Directors or Certain O...,pos
5677,Item 2.02 Results of Operations and Financial ...,neg


In [24]:
tt1.to_csv('text_train.csv')

In [26]:
# get a test set data frame of texts and labels only
tt2 = text_test.to_frame().rename({'texts': 'text'}, axis=1)
tt2.loc[:, 'label'] = y_test

In [27]:
tt2.head()

,text,label
9723,Item 2.02. Results of Operations and Financial...,neg
19515,Item 5.07\nSubmission of Matters to a Vote of ...,neg
11724,Item 5.02. Departure of Directors or Certain O...,neg
2351,Item 5.02.Departure of Directors or Certain Of...,neg
18194,Item 5.02\nDeparture of Directors or Certain O...,pos


In [28]:
tt2.to_csv('text_test.csv')

In [29]:
num_train.shape, text_train.shape

((17361, 70), (17361,))

In [30]:
import torch
import torchtext
from torchtext.legacy.data import Field, LabelField, TabularDataset
from torch.utils.data import DataLoader, TensorDataset
from torchtext.legacy import data as dl
import torch.nn as nn
import torch.optim as optim

In [31]:
# create train and validation data loader for numerical features (set shuffle to False to keep same indices with texts)
dataloader2 = DataLoader(TensorDataset(torch.from_numpy(num_train.values)), batch_size=64, shuffle=False)
dataloader2_valid = DataLoader(TensorDataset(torch.from_numpy(num_test.values)), batch_size=64, shuffle=False)

In [33]:
# define iterators
dataloader2_iter = dataloader2._get_iterator()
dataloader2_valid_iter = dataloader2_valid._get_iterator()

In [34]:
# define the fields with pre-processing pipeline
TEXT = Field(tokenize="spacy",lower = True, sequential=True, batch_first=True,include_lengths=True)
LABEL = LabelField(dtype = torch.float,batch_first=True)

In [35]:
# define columns names (None is the ID column)
fields = [(None, None),('text',TEXT), ('label', LABEL)]

In [37]:
# load and pre-process texts and labels
train_data=TabularDataset(path = "text_train.csv",format = 'csv',fields = fields,skip_header = True,)
valid_data=TabularDataset(path = "text_test.csv",format = 'csv',fields = fields,skip_header = True,)

In [41]:
# build vocabulary with training set using 100d pre-trained GloVe embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:42, 5.31MB/s]                           
 99%|█████████▉| 397785/400000 [00:14<00:00, 28365.48it/s]

In [43]:
print(len(TEXT.vocab))
print("---------------")
print(len(LABEL.vocab))
#print("---------------", "\n")
#print(TEXT.vocab.freqs.most_common(100)) 
#print("---------------", "\n")
#print(TEXT.vocab.stoi) 
#print("---------------", "\n")
#print(LABEL.vocab.stoi)

22580
---------------
2


In [44]:
# create iterators
iterator1 = dl.Iterator(train_data, batch_size=64, shuffle=False)
iterator1_valid = dl.Iterator(valid_data, batch_size=64, shuffle=False)

In [45]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, aux_dim, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True)

        #dense layer
        self.fc = nn.Linear(hidden_dim * 2 + aux_dim, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths, features):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(),batch_first=True, enforce_sorted=False).cuda()
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

        #concat the hidden state with numerical features

        out1 = torch.cat([hidden[:features.shape[0], :], features], dim=1).float()
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(out1)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [46]:
aux_dim = dataloader2_iter.__next__()[0].shape[-1]
aux_dim

70

In [55]:
# define the device
device = torch.device('cuda')

#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 64
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2
aux_dim = aux_dim
#instantiate the model
model = classifier(size_of_vocab, aux_dim, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers, 
                   bidirectional = True, dropout = dropout).to(device)

In [56]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(22580, 100)
  (lstm): LSTM(100, 64, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=198, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 2,442,519 trainable parameters
torch.Size([22580, 100])


In [ ]:
pip install transformers

In [57]:
import torch.optim as optim
from transformers import AdamW

#define optimizer and loss
optimizer = AdamW(model.parameters(), lr = 1e-4)
criterion = nn.BCELoss()
    
#push to cuda
criterion = criterion.to(device)

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc


In [58]:
def train(model, iterator1, iterator2, optimizer, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch1 in iterator1:

        #call feature iterator
        batch2 = iterator2.__next__()
        
        #resets the gradients after every batch
        optimizer.zero_grad()   
        
        #retrieve texts
        text, text_lengths = batch1.text   
        
        #get predictions
        predictions = model(text.to(device), text_lengths.to(device), batch2[0].to(device)).squeeze()
        
        #compute the loss
        loss = criterion(predictions, batch1.label.to(device))
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch1.label.to(device))   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimizer.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator1), epoch_acc / len(iterator1)

In [59]:
def evaluate(model, iterator1, iterator2, criterion):
    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch1 in iterator1:
        
            #retrieve texts
            text, text_lengths = batch1.text

            #call feature iterator
            batch2 = iterator2.__next__()
            
            #get predictions
            predictions = model(text.to(device), text_lengths.to(device), batch2[0].to(device)).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch1.label.to(device))
            acc = binary_accuracy(predictions, batch1.label.to(device))
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator1), epoch_acc / len(iterator1)

In [60]:
N_EPOCHS = 25
best_valid_loss = float('inf')
best_valid_accuracy = 0

for epoch in range(N_EPOCHS):
     
    #train the model
    dataloader2_iter = dataloader2._get_iterator()
    dataloader2_valid_iter = dataloader2_valid._get_iterator()
    train_loss, train_acc = train(model, iterator1, dataloader2_iter, optimizer, criterion)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(model, iterator1_valid, dataloader2_valid_iter, criterion)
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    if epoch%5 == 0:
      print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
      print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
  
    if valid_acc > best_valid_accuracy:
      best_valid_accuracy = valid_acc

	Train Loss: 0.691 | Train Acc: 54.05%
	 Val. Loss: 0.691 |  Val. Acc: 55.38%
	Train Loss: 0.678 | Train Acc: 57.30%
	 Val. Loss: 0.688 |  Val. Acc: 56.82%
	Train Loss: 0.658 | Train Acc: 60.66%
	 Val. Loss: 0.698 |  Val. Acc: 57.10%
	Train Loss: 0.635 | Train Acc: 63.76%
	 Val. Loss: 0.717 |  Val. Acc: 56.22%
	Train Loss: 0.604 | Train Acc: 66.94%
	 Val. Loss: 0.752 |  Val. Acc: 56.12%


In [61]:
best_valid_accuracy

0.5749124235966626